In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from collections import defaultdict

import warnings; warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)


In [2]:
credits = pd.read_csv('C:\\Datasets\\credits.csv')
keywords = pd.read_csv('C:\\Datasets\\keywords.csv')
links_small = pd.read_csv('C:\\Datasets\\links_small.csv')
movies_metadata = pd.read_csv('C:\\Datasets\\movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('C:\\Datasets\\ratings_small.csv')

In [3]:
credits.head()

cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'}, {'cast_id': 19, 'character': 'Hamm (voice)', 'credit_id': '52fe4284c3a36847f8024fa9', 'gender': 2, 'id': 7907, 'name': 'John Ratzenberger', 'order': 5, 'profile_path': '/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg'}, {'cast_id': 20, 'character': 'Bo Peep (voice)', 'credit_id': '52fe4284c3a36847f8024fad', 'gender': 1, 'id': 8873, 'name': 'Annie Potts', 'order': 6, 'profile_path': '/eryXT84RL41jHSJcMy4kS3u9y6w.jpg'}, {'cast_id': 26, 'character': 'Andy (voice)', 'credit_id': '52fe4284c3a36847f8024fc1', 'gender': 0, 'id': 1116442, 'name': 'John Morris', 'order': 7, 'profile_path': '/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg'}, {'cast_id': 22, 'character': 'Sid (voice)', 'credit_id': '52fe4284c3a36847f8024fb1', 'gender': 2, 'id': 12901, 'name': 'Erik von Detten', 'order': 8, 'profile_path': '/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg'}, {'cast_id': 23, 'character': 'Mrs. Davis (voice)', 'credit_id': '52fe4284c3a36847f8024fb5', 'gender': 1, 'id': 12133, 'name': 'Laurie Metcalf', 'order': 9, 'profile_path': '/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg'}, {'cast_id': 24, 'character': 'Sergeant (voice)', 'credit_id': '52fe4284c3a36847f8024fb9', 'gender': 2, 'id': 8655, 'name': 'R. Lee Ermey', 'order': 10, 'profile_path': '/r8GBqFBjypLUP9VVqDqfZ7wYbSs.jpg'}, {'cast_id': 25, 'character': 'Hannah (voice)', 'credit_id': '52fe4284c3a36847f8024fbd', 'gender': 1, 'id': 12903, 'name': 'Sarah Freeman', 'order': 11, 'profile_path': None}, {'cast_id': 27, 'character': 'TV Announcer (voice)', 'credit_id': '52fe4284c3a36847f8024fc5', 'gender': 2, 'id': 37221, 'name': 'Penn Jillette', 'order': 12, 'profile_path': '/zmAaXUdx12NRsssgHbk1T31j2x9.jpg'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [4]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 10941, 'name': 'disappearance'}, {'id': 15101, 'name': ""based on children's book""}, {'id': 33467, 'name': 'new home'}, {'id': 158086, 'name': 'recluse'}, {'id': 158091, 'name': 'giant insect'}]"
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392, 'name': 'best friend'}, {'id': 179431, 'name': 'duringcreditsstinger'}, {'id': 208510, 'name': 'old men'}]"
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id': 10131, 'name': 'interracial relationship'}, {'id': 14768, 'name': 'single mother'}, {'id': 15160, 'name': 'divorce'}, {'id': 33455, 'name': 'chick flick'}]"
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'name': 'midlife crisis'}, {'id': 2246, 'name': 'confidence'}, {'id': 4995, 'name': 'aging'}, {'id': 5600, 'name': 'daughter'}, {'id': 10707, 'name': 'mother daughter relationship'}, {'id': 13149, 'name': 'pregnancy'}, {'id': 33358, 'name': 'contraception'}, {'id': 170521, 'name': 'gynecologist'}]"


In [5]:
links_small.head(20)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
5,6,113277,949.0
6,7,114319,11860.0
7,8,112302,45325.0
8,9,114576,9091.0
9,10,113189,710.0


In [6]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg'

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [8]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [9]:
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [10]:
movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [11]:
movies_metadata.iloc[0:2].transpose()

,0,1
adult,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",NaN
budget,30000000,65000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]","[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"
homepage,http://toystory.disney.com/toy-story,NaN
id,862,8844
imdb_id,tt0114709,tt0113497
original_language,en,en
original_title,Toy Story,Jumanji
overview,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.","When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."


In [12]:
movies_metadata['genres'] = movies_metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i[
    'name'] for i in x] if isinstance(x, list) else [])

In [13]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, 

In [14]:
movies_metadata['year'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [15]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ...

In [16]:
movies_metadata.iloc[0:3].transpose()

,0,1,2
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}"
budget,30000000,65000000,0
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN
id,862,8844,15602
imdb_id,tt0114709,tt0113497,tt0113228
original_language,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men
overview,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.","When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.","A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max."


In [17]:
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'year'],
      dtype='object')

In [18]:
movies_metadata[movies_metadata['id']=='1997-08-20']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[Carousel Productions, Vision View Entertainment, Telescene Film Group Productions]","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'LU', 'name': 'Luxembourg'}, {'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [19]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [20]:
## Pre-processing step

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [21]:
movies_metadata['id'] = movies_metadata['id'].apply(convert_int)
movies_metadata[movies_metadata['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[Carousel Productions, Vision View Entertainment, Telescene Film Group Productions]","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'LU', 'name': 'Luxembourg'}, {'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
29503,Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[Aniplex, GoHands, BROSTA TV, Mardock Scramble Production Committee, Sentai Filmworks]","[{'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'JP', 'name': 'Japan'}]",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
35587,Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[Odyssey Media, Pulser Productions, Rogue State, The Cartel]","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,False,4.3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [22]:
movies_metadata = movies_metadata.drop([19730, 29503, 35587])

In [23]:
movies_metadata['id'] = movies_metadata['id'].astype('int')

In [24]:
shape_movies_metadata = movies_metadata[movies_metadata['id'].isin(links_small)]
shape_movies_metadata.shape

(9099, 25)

In [25]:
shape_movies_metadata['tagline'] = shape_movies_metadata['tagline'].fillna('')
shape_movies_metadata['description'] = shape_movies_metadata['overview'] + shape_movies_metadata['tagline']
shape_movies_metadata['description'] = shape_movies_metadata['description'].fillna('')

In [26]:
shape_movies_metadata['description'].head(10)

0    Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.                                                                                                                                                                           
1    When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.Roll the dice and unleash the excitement!                                      
2    A family wedding reignites the anci

In [27]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(shape_movies_metadata['description'])

In [28]:
tfidf_matrix.shape

(9099, 268124)

In [29]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [30]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [31]:
shape_movies_metadata = shape_movies_metadata.reset_index()
titles = shape_movies_metadata['title']
indices = pd.Series(shape_movies_metadata.index, index=shape_movies_metadata['title'])

In [32]:
titles.head(20)

0     Toy Story                     
1     Jumanji                       
2     Grumpier Old Men              
3     Waiting to Exhale             
4     Father of the Bride Part II   
5     Heat                          
6     Sabrina                       
7     Tom and Huck                  
8     Sudden Death                  
9     GoldenEye                     
10    The American President        
11    Dracula: Dead and Loving It   
12    Balto                         
13    Nixon                         
14    Cutthroat Island              
15    Casino                        
16    Sense and Sensibility         
17    Four Rooms                    
18    Ace Ventura: When Nature Calls
19    Money Train                   
Name: title, dtype: object

In [33]:
def get_recommendations(title, num):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num+1]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [34]:
get_recommendations('Pulp Fiction',10)

7977    Holding Trevor                        
6974    The Wackness                          
2935    Prizzi's Honor                        
6734    You Kill Me                           
240     Little Odessa                         
562     Moll Flanders                         
6837    Wristcutters: A Love Story            
5416    La Jetée                              
7433    The First Day of the Rest of Your Life
8739    Comet                                 
Name: title, dtype: object

In [35]:
get_recommendations('Frozen',10)

3764    Serendipity                     
8498    Winter's Tale                   
6781    Lars and the Real Girl          
4911    Hercules in New York            
115     Pie in the Sky                  
3447    How to Marry a Millionaire      
4105    Spirit: Stallion of the Cimarron
7406    Leap Year                       
1653    Sleeping Beauty                 
4951    What's New Pussycat?            
Name: title, dtype: object

In [36]:
credits.head(20)

cast  \
0   [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'}, {'cast_id': 19, 'character': 'Hamm (voice)', 'credit_id': '52fe4284c3a36847f8024fa9', 'gender': 2, 'id': 7907, 'name': 'John Ratzenberger', 'order': 5, 'profile_path': '/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg'}, {'cast_id': 20, 'character': 'Bo Peep (voice)', 'credit_id': '52fe4284c3a36847f8024fad', 'gender': 1, 'id': 8873, 'name': 'Annie Potts', 'order': 6, 'profile_path': '/eryXT84RL41jHSJcMy4kS3u9y6w.jpg'}, {'cast_id': 26, 'character': 'Andy (voice)', 'credit_id': '52fe4284c3a36847f8024fc1', 'gender': 0, 'id': 1116442, 'name': 'John Morris', 'order': 7, 'profile_path': '/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg'}, {'cast_id': 22, 'character': 'Sid (voice)', 'credit_id': '52fe4284c3a36847f8024fb1', 'gender': 2, 'id': 12901, 'name': 'Erik von Detten', 'order': 8, 'profile_path': '/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg'}, {'cast_id': 23, 'character': 'Mrs. Davis (voice)', 'credit_id': '52fe4284c3a36847f8024fb5', 'gender': 1, 'id': 12133, 'name': 'Laurie Metcalf', 'order': 9, 'profile_path': '/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg'}, {'cast_id': 24, 'character': 'Sergeant (voice)', 'credit_id': '52fe4284c3a36847f8024fb9', 'gender': 2, 'id': 8655, 'name': 'R. Lee Ermey', 'order': 10, 'profile_path': '/r8GBqFBjypLUP9VVqDqfZ7wYbSs.jpg'}, {'cast_id': 25, 'character': 'Hannah (voice)', 'credit_id': '52fe4284c3a36847f8024fbd', 'gender': 1, 'id': 12903, 'name': 'Sarah Freeman', 'order': 11, 'profile_path': None}, {'cast_id': 27, 'character': 'TV Announcer (voice)', 'credit_id': '52fe4284c3a36847f8024fc5', 'gender': 2, 'id': 37221, 'name': 'Penn Jillette', 'order': 12, 'profile_path': '/zmAaXUdx12NRsssgHbk1T31j2x9.jpg'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [37]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies_metadata['id'] = movies_metadata['id'].astype('int')

In [38]:
credits['cast'].head(20)

0     [{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP

In [39]:
movies_metadata.head(20)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ...

In [40]:
movies_metadata = movies_metadata.merge(credits, on='id')
movies_metadata = movies_metadata.merge(keywords, on='id')

In [41]:
shape_movies_metadata = movies_metadata[movies_metadata['id'].isin(links_small)]
shape_movies_metadata.shape

(9219, 28)

In [42]:
shape_movies_metadata['cast'] = shape_movies_metadata['cast'].apply(literal_eval)
shape_movies_metadata['crew'] = shape_movies_metadata['crew'].apply(literal_eval)
shape_movies_metadata['keywords'] = shape_movies_metadata['keywords'].apply(literal_eval)
shape_movies_metadata['cast_size'] = shape_movies_metadata['cast'].apply(lambda x: len(x))
shape_movies_metadata['crew_size'] = shape_movies_metadata['crew'].apply(lambda x: len(x))

In [43]:
shape_movies_metadata.head().transpose()

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",NaN,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}"
budget,30000000,65000000,0,16000000,0
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]","[Comedy, Drama, Romance]",[Comedy]
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.","When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.","A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.","Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.","Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own."


In [44]:
shape_movies_metadata['crew'].head(5)

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}, {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}, {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}, {'credit_id': '52fe4284c3a36847f8024f67', 'd

In [45]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [46]:
def get_producer(x):
    for i in x:
        if i['job'] == 'Producer':
            return i['name']
    return np.nan

In [47]:
shape_movies_metadata['director'] = shape_movies_metadata['crew'].apply(get_director)

In [48]:
shape_movies_metadata['producer'] = shape_movies_metadata['crew'].apply(get_producer)

In [49]:
shape_movies_metadata[['director','producer']].head(-10)

,director,producer
0,John Lasseter,Bonnie Arnold
1,Joe Johnston,Scott Kroopf
2,Howard Deutch,NaN
3,Forest Whitaker,Ronald Bass
4,Charles Shyer,Nancy Meyers
...,...,...
40528,Sian Heder,Chris Columbus
40554,Alan Barillaro,Marc Sondheimer
40631,Henry Joost,Anthony Katagas
40688,Steffi Kammerer,NaN


In [50]:
shape_movies_metadata['cast'] = shape_movies_metadata['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
shape_movies_metadata['cast'] = shape_movies_metadata['cast'].apply(lambda x: x[:5] if len(x) >=5 else x)

In [51]:
shape_movies_metadata['cast'].head(20)

0     [Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn]                          
1     [Robin Williams, Jonathan Hyde, Kirsten Dunst, Bradley Pierce, Bonnie Hunt]             
2     [Walter Matthau, Jack Lemmon, Ann-Margret, Sophia Loren, Daryl Hannah]                  
3     [Whitney Houston, Angela Bassett, Loretta Devine, Lela Rochon, Gregory Hines]           
4     [Steve Martin, Diane Keaton, Martin Short, Kimberly Williams-Paisley, George Newbern]   
5     [Al Pacino, Robert De Niro, Val Kilmer, Jon Voight, Tom Sizemore]                       
6     [Harrison Ford, Julia Ormond, Greg Kinnear, Angie Dickinson, Nancy Marchand]            
7     [Jonathan Taylor Thomas, Brad Renfro, Rachael Leigh Cook, Michael McShane, Amy Wright]  
8     [Jean-Claude Van Damme, Powers Boothe, Dorian Harewood, Raymond J. Barry, Ross Malinger]
9     [Pierce Brosnan, Sean Bean, Izabella Scorupco, Famke Janssen, Joe Don Baker]            
10    [Michael Douglas, Annette Bening, Michael J.

In [52]:
shape_movies_metadata['keywords'] = shape_movies_metadata['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [53]:
shape_movies_metadata['cast'] = shape_movies_metadata['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [54]:
shape_movies_metadata['director'] = shape_movies_metadata['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [55]:
shape_movies_metadata['producer'] = shape_movies_metadata['producer'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [56]:
shape_movies_metadata['producer'] = shape_movies_metadata['producer'].apply(lambda x: [x])

In [57]:
shape_movies_metadata['director'] = shape_movies_metadata['director'].apply(lambda x: [x])

In [58]:
s = shape_movies_metadata.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [59]:
s = s.value_counts()

In [60]:
s.head(20)

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
violence                264
love                    222
musical                 219
sex                     219
suspense                212
revenge                 208
friendship              202
nudity                  197
police                  195
sport                   192
dystopia                184
aftercreditsstinger     178
biography               178
sequel                  160
prison                  155
Name: keyword, dtype: int64

In [61]:
s = s[s > 1]

In [62]:
stemmer = SnowballStemmer('english')
stemmer.stem('chairs')

'chair'

In [63]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [64]:
shape_movies_metadata['keywords'] = shape_movies_metadata['keywords'].apply(filter_keywords)
shape_movies_metadata['keywords'] = shape_movies_metadata['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
shape_movies_metadata['keywords'] = shape_movies_metadata['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [65]:
shape_movies_metadata['keywords'].head(20)

0     [jealousi, toy, boy, friendship, friend, rivalri, boynextdoor, newtoy, toycomestolif]                                                                                                                                
1     [boardgam, disappear, basedonchildren'sbook, newhom, reclus, giantinsect]                                                                                                                                            
2     [fish, bestfriend, duringcreditssting]                                                                                                                                                                               
3     [basedonnovel, interracialrelationship, singlemoth, divorc, chickflick]                                                                                                                                              
4     [babi, midlifecrisi, confid, age, daughter, motherdaughterrelationship, pregnanc, contracept, gynecologist]       

In [66]:
shape_movies_metadata['cast'].head(20)

0     [tomhanks, timallen, donrickles, jimvarney, wallaceshawn]                        
1     [robinwilliams, jonathanhyde, kirstendunst, bradleypierce, bonniehunt]           
2     [waltermatthau, jacklemmon, ann-margret, sophialoren, darylhannah]               
3     [whitneyhouston, angelabassett, lorettadevine, lelarochon, gregoryhines]         
4     [stevemartin, dianekeaton, martinshort, kimberlywilliams-paisley, georgenewbern] 
5     [alpacino, robertdeniro, valkilmer, jonvoight, tomsizemore]                      
6     [harrisonford, juliaormond, gregkinnear, angiedickinson, nancymarchand]          
7     [jonathantaylorthomas, bradrenfro, rachaelleighcook, michaelmcshane, amywright]  
8     [jean-claudevandamme, powersboothe, dorianharewood, raymondj.barry, rossmalinger]
9     [piercebrosnan, seanbean, izabellascorupco, famkejanssen, joedonbaker]           
10    [michaeldouglas, annettebening, michaelj.fox, martinsheen, annadeaveresmith]     
11    [leslienielsen, melbrooks,

In [67]:
shape_movies_metadata['director'].head(20)

0     [johnlasseter]  
1     [joejohnston]   
2     [howarddeutch]  
3     [forestwhitaker]
4     [charlesshyer]  
5     [michaelmann]   
6     [sydneypollack] 
7     [peterhewitt]   
8     [peterhyams]    
9     [martincampbell]
10    [robreiner]     
11    [melbrooks]     
12    [simonwells]    
13    [oliverstone]   
14    [rennyharlin]   
15    [martinscorsese]
16    [anglee]        
17    [allisonanders] 
18    [steveoedekerk] 
19    [josephruben]   
Name: director, dtype: object

In [68]:
shape_movies_metadata['producer'].head(20)

0     [bonniearnold]   
1     [scottkroopf]    
2     [nan]            
3     [ronaldbass]     
4     [nancymeyers]    
5     [artlinson]      
6     [sydneypollack]  
7     [nan]            
8     [moshediamant]   
9     [barbarabroccoli]
10    [robreiner]      
11    [melbrooks]      
12    [stevenspielberg]
13    [danhalsted]     
14    [joelb.michaels] 
15    [barbaradefina]  
16    [lindsaydoran]   
17    [lawrencebender] 
18    [bobisrael]      
19    [neilcanton]     
Name: producer, dtype: object

In [69]:
shape_movies_metadata['genres'].head(20)

0     [Animation, Comedy, Family]       
1     [Adventure, Fantasy, Family]      
2     [Romance, Comedy]                 
3     [Comedy, Drama, Romance]          
4     [Comedy]                          
5     [Action, Crime, Drama, Thriller]  
6     [Comedy, Romance]                 
7     [Action, Adventure, Drama, Family]
8     [Action, Adventure, Thriller]     
9     [Adventure, Action, Thriller]     
10    [Comedy, Drama, Romance]          
11    [Comedy, Horror]                  
12    [Family, Animation, Adventure]    
13    [History, Drama]                  
14    [Action, Adventure]               
15    [Drama, Crime]                    
16    [Drama, Romance]                  
17    [Crime, Comedy]                   
18    [Crime, Comedy, Adventure]        
19    [Action, Comedy, Crime]           
Name: genres, dtype: object

In [70]:
shape_movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9219 entries, 0 to 41669
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9219 non-null   object 
 1   belongs_to_collection  1688 non-null   object 
 2   budget                 9219 non-null   object 
 3   genres                 9219 non-null   object 
 4   homepage               2001 non-null   object 
 5   id                     9219 non-null   int32  
 6   imdb_id                9219 non-null   object 
 7   original_language      9219 non-null   object 
 8   original_title         9219 non-null   object 
 9   overview               9207 non-null   object 
 10  popularity             9219 non-null   object 
 11  poster_path            9216 non-null   object 
 12  production_companies   9219 non-null   object 
 13  production_countries   9219 non-null   object 
 14  release_date           9219 non-null   object 
 15  rev

In [71]:
shape_movies_metadata['collect'] = shape_movies_metadata['keywords'] + shape_movies_metadata['cast'] 
+ shape_movies_metadata['genres'] 
+ shape_movies_metadata['director'] + shape_movies_metadata['producer']

0        [johnlasseter, bonniearnold]          
1        [joejohnston, scottkroopf]            
2        [howarddeutch, nan]                   
3        [forestwhitaker, ronaldbass]          
4        [charlesshyer, nancymeyers]           
                    ...                        
40952    [greggchampion, nan]                  
41172    [tinusureshdesai, neerajpandey]       
41225    [ashutoshgowariker, ashutoshgowariker]
41391    [hideakianno, yoshihirosatô]          
41669    [ronhoward, briangrazer]              
Length: 9219, dtype: object

In [72]:
shape_movies_metadata['collect'] = shape_movies_metadata['collect'].apply(lambda x: ' '.join(x))

In [73]:
shape_movies_metadata['collect'].head(20)

0     jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif tomhanks timallen donrickles jimvarney wallaceshawn                                                                                                                   
1     boardgam disappear basedonchildren'sbook newhom reclus giantinsect robinwilliams jonathanhyde kirstendunst bradleypierce bonniehunt                                                                                                               
2     fish bestfriend duringcreditssting waltermatthau jacklemmon ann-margret sophialoren darylhannah                                                                                                                                                   
3     basedonnovel interracialrelationship singlemoth divorc chickflick whitneyhouston angelabassett lorettadevine lelarochon gregoryhines                                                                                                              
4   

In [74]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(shape_movies_metadata['collect'])

In [75]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [76]:
shape_movies_metadata = shape_movies_metadata.reset_index()
titles = shape_movies_metadata['title']
indices = pd.Series(shape_movies_metadata.index, index=shape_movies_metadata['title'])

In [77]:
get_recommendations('Pulp Fiction',10)

7831    The Sunset Limited        
616     The Great White Hype      
9038    Afro Samurai              
440     Menace II Society         
231     Kiss of Death             
3062    White Sands               
6939    Cleaner                   
7148    Soul Men                  
7233    Afro Samurai: Resurrection
7868    African Cats              
Name: title, dtype: object

In [78]:
get_recommendations('Afro Samurai: Resurrection',10)

7831    The Sunset Limited  
9038    Afro Samurai        
231     Kiss of Death       
3062    White Sands         
6939    Cleaner             
7148    Soul Men            
7868    African Cats        
8558    Oldboy              
9107    Strictly Business   
616     The Great White Hype
Name: title, dtype: object

In [79]:
reader = Reader()

In [80]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [81]:
algo =SVD()

In [82]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [83]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8934  0.8922  0.8988  0.8986  0.9040  0.8974  0.0042  
MAE (testset)     0.6858  0.6857  0.6933  0.6932  0.6958  0.6907  0.0042  
Fit time          4.54    3.96    4.99    5.29    4.66    4.69    0.45    
Test time         0.15    0.12    0.16    0.16    0.10    0.14    0.02    


{'test_rmse': array([0.89343149, 0.89221647, 0.89875731, 0.8986146 , 0.90395552]),
 'test_mae': array([0.68582875, 0.68566504, 0.69326847, 0.69316311, 0.69578634]),
 'fit_time': (4.544194459915161,
  3.964782238006592,
  4.993258476257324,
  5.2939934730529785,
  4.659204721450806),
 'test_time': (0.14686322212219238,
  0.11900973320007324,
  0.16260862350463867,
  0.1579761505126953,
  0.09868049621582031)}

In [84]:
ratings[ratings['userId'] == 26]

,userId,movieId,rating,timestamp
4781,26,1,5.0,1360087980
4782,26,32,4.5,1354313537
4783,26,47,4.5,1360088070
4784,26,50,4.5,1360088047
4785,26,63,0.5,1351544773
...,...,...,...,...
4948,26,79132,3.5,1354313972
4949,26,80489,2.0,1354313516
4950,26,82459,4.0,1351544798
4951,26,91542,3.5,1351545019


In [85]:
trainset = data.build_full_trainset()
algo.fit(trainset)

In [86]:
algo.predict(26,15,4)

Prediction(uid=26, iid=15, r_ui=4, est=2.2067875637418335, details={'was_impossible': False})

In [87]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [88]:
links_small.head(20)

0     862  
1     8844 
2     15602
3     31357
4     11862
5     949  
6     11860
7     45325
8     9091 
9     710  
10    9087 
11    12110
12    21032
13    10858
14    1408 
15    524  
16    4584 
17    5    
18    9273 
19    11517
Name: tmdbId, dtype: int32

In [89]:
id_map = pd.read_csv('C:\\Datasets\\links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(shape_movies_metadata[['title', 'id']], on='id').set_index('title')

In [90]:
id_map.head()

,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0


In [91]:
indices_map = id_map.set_index('id')

In [92]:
indices_map.head()

,movieId
id,
862.0,1
8844.0,2
15602.0,3
31357.0,4
11862.0,5


In [93]:
def hybridrec(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:50]
    movie_indices = [i[0] for i in sim_scores]
    movies = shape_movies_metadata.iloc[movie_indices][['title', 'vote_average', 'id']]
    movies['est rating'] = movies['id'].apply(lambda x: algo.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est rating', ascending=False)
    return movies.head(10)

In [94]:
hybridrec(1, 'Pulp Fiction')

,title,vote_average,id,est rating
5686,The Incredibles,7.4,9806,3.112744
8846,Kingsman: The Secret Service,7.6,207703,2.985516
4306,The 51st State,5.9,1613,2.896401
8905,The Hateful Eight,7.6,273248,2.888640
4084,Changing Lanes,5.9,1537,2.829331
7831,The Sunset Limited,7.1,56831,2.826442
9200,The Legend of Tarzan,5.5,258489,2.796867
6293,The Man,5.4,9074,2.795534
405,Fresh,7.6,13815,2.790231
144,Die Hard: With a Vengeance,6.9,1572,2.757792


In [95]:
hybridrec(2, 'Afro Samurai')

,title,vote_average,id,est rating
8905,The Hateful Eight,7.6,273248,3.901415
6199,Star Wars: Episode III - Revenge of the Sith,7.1,1895,3.762423
1636,The Negotiator,6.8,9631,3.730580
8846,Kingsman: The Secret Service,7.6,207703,3.728962
2387,Hard Eight,6.8,8052,3.683763
2745,School Daze,6.1,36739,3.677856
6293,The Man,5.4,9074,3.674446
5686,The Incredibles,7.4,9806,3.652451
440,Menace II Society,7.2,9516,3.598104
4084,Changing Lanes,5.9,1537,3.596690


In [96]:
#smd['collect'] = smd['keywords'] + smd['cast'] + smd['genres'] + smd['director'] + smd['producer']
#    jealousi toy boy friendship oldfriend rivalri story boynextdoor newtoy toycomestolif tomhanks timallen anniepotts tonyhale keeganmichaelkey Animation Comedy Family joshcooley marknielsen                                                                                                                                                                                                     
#count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
#count_matrix = count.fit_transform(smd['collect'])
#cosine_sim = cosine_similarity(count_matrix, count_matrix)
#smd = smd.reset_index()
#titles = smd['title']
#indices = pd.Series(smd.index, index=smd['title'])

In [97]:
#Let's assume Toy Story 4 is released and there are no votes/ratings yet so it is a cold item
# the details of the movie except the ratings, popularity, etc. would be made available so we are mocking that
# and adding the details manually
shape_movies_metadata.loc[len(shape_movies_metadata.index)] = [5989384,0, 0, 10000,'Animation','','301528','tt1979376','en','Toy Story 4','','','','','','2019',0,125
                          ,'','','','Toy Story 4','',0,0,'2019','','','','','','','',
                          'jealousi toy boy friendship oldfriend rivalri story boynextdoor newtoy toycomestolif tomhanks timallen anniepotts tonyhale keeganmichaelkey Animation Comedy Family joshcooley marknielsen'] 


In [98]:
shape_movies_metadata.tail(1)

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,producer,collect
9219,5989384,0,0,10000,Animation,,301528,tt1979376,en,Toy Story 4,,,,,,2019,0.0,125.0,,,,Toy Story 4,,0.0,0.0,2019,,,,,,,,jealousi toy boy friendship oldfriend rivalri story boynextdoor newtoy toycomestolif tomhanks timallen anniepotts tonyhale keeganmichaelkey Animation Comedy Family joshcooley marknielsen


In [99]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(shape_movies_metadata['collect'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
shape_movies_metadata = shape_movies_metadata.reset_index()
titles = shape_movies_metadata['title']
indices = pd.Series(shape_movies_metadata.index, index=shape_movies_metadata['title'])

In [100]:
# the idea is if the ratings is missing as would in the case of a new released movie, we would see if the closest 
# similar movie could act as a proxy to get the estimated ratings to recommend the new movie to user
def hybridrecsolvecolditem(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:50]
    movie_indices = [i[0] for i in sim_scores]
    movies = shape_movies_metadata.iloc[movie_indices][['title', 'vote_average', 'id']]
    movies['est rating']= 0
    moviesfinal = pd.DataFrame(columns=movies.columns)
    for index, row in movies.iterrows():
        # this tells us if this this a new movie
        if row['title'] not in id_map.index:
            title1=row['title']
            idx1 = indices[title1]
            sim_scores1 = list(enumerate(cosine_sim[int(idx1)]))
            sim_scores1 = sorted(sim_scores1, key=lambda x: x[1], reverse=True)
            sim_scores1 = sim_scores1[1:2]
            movie_indices1 = [i[0] for i in sim_scores1]
            #this movie1 is actually a proxy for the suggested movie
            movies1 = shape_movies_metadata.iloc[movie_indices1][['title', 'vote_average', 'id']]
            movies1['title']=title1
            movies1['est rating'] = movies1['id'].apply(lambda x: algo.predict(userId, indices_map.loc[x]['movieId']).est)
            movies1['vote_average']=row['vote_average']
            movies1['id']=row['id']
            moviesfinal=moviesfinal.append(movies1)
        else:
            movies2 = pd.DataFrame(columns=movies.columns)
            movies2 = movies2.append(row)
            movies2['est rating'] = movies2['id'].apply(lambda x: algo.predict(userId, indices_map.loc[x]['movieId']).est)
            moviesfinal=moviesfinal.append(movies2)
    moviesfinal = moviesfinal.sort_values('est rating', ascending=False)
    return (moviesfinal.head(10))

In [101]:
hybridrecsolvecolditem(2,'Toy Story')

,title,vote_average,id,est rating
1030,Stand by Me,7.8,235,4.224861
7500,Hachi: A Dog's Tale,7.7,28178,4.058215
7629,Toy Story 3,7.6,10193,3.984213
2817,The Odd Couple,7.5,11356,3.976829
321,Forrest Gump,8.2,13,3.976699
7340,(500) Days of Summer,7.2,19913,3.946848
2258,Big,6.9,2280,3.851897
3438,Startup.com,6.3,14268,3.814165
8007,Hugo,7.0,44826,3.809663
8595,The Lego Movie,7.5,137106,3.793070


In [102]:
hybridrecsolvecolditem(1,'Toy Story')

,title,vote_average,id,est rating
1030,Stand by Me,7.8,235,3.421128
7629,Toy Story 3,7.6,10193,3.412845
8310,Django Unchained,7.8,68718,3.373864
2817,The Odd Couple,7.5,11356,3.189973
8595,The Lego Movie,7.5,137106,3.067223
3833,"Monsters, Inc.",7.5,585,3.066229
3438,Startup.com,6.3,14268,3.044625
2258,Big,6.9,2280,3.030992
7500,Hachi: A Dog's Tale,7.7,28178,2.994697
2206,Jules and Jim,7.6,1628,2.943201


In [103]:
algo.predict(2,23146737967567)

Prediction(uid=2, iid=23146737967567, r_ui=None, est=3.455398717131989, details={'was_impossible': False})

In [104]:
#ratings.loc[ratings['movieId']=='213121']
ratings.head(-1000)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
98999,664,7438,4.0,1343746819
99000,664,7445,4.5,1343747211
99001,664,7458,3.0,1393891196
99002,664,7502,4.5,1343731683


In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=4):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.6046944858420279
0.23045048514377217
0.6083954297069062
0.2342930209603063
0.623298559364134
0.2406202594419601
0.6087064676616925
0.22652036069538242
